In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/3.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/4.csv"), ignore_index=True)
print(len(postulaciones2))
postulaciones2.head()

13209068


,idaviso,idpostulante,sepostulo
0,1112257047,ow2xPaj,0
1,1111920714,LNKXqJB,0
2,1112346945,8MBPqYD,0
3,1112345547,ek4P2Wr,0
4,1112237522,NpqMD5,0


In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [8]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

19813602
19722959


In [9]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
8550737,1112115684,2zwm9vQ,0,MASC,28.0,-,-,-,En Curso,Graduado,...,False,False,False,False,False,False,False,True,False,NaN
2555601,1112260365,owaA9Gk,1,MASC,19.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,True,NaN
774320,1112412960,akjG8mE,0,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
4332701,1112424481,ZDrXXD8,0,FEM,20.0,-,-,-,-,-,...,False,False,True,False,False,False,False,True,False,NaN
16005151,1112227446,N0w6p5,0,FEM,38.0,-,-,Graduado,Graduado,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
8015724,1112430277,6rZ8WOv,1,FEM,20.0,-,-,En Curso,-,-,...,False,False,False,False,False,False,False,False,False,NaN
9530866,1112303993,Bm51vVR,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
17376001,1112452624,eBvZ69,0,MASC,27.0,-,Graduado,Graduado,Graduado,-,...,False,True,False,False,False,False,False,False,False,NaN
11747878,1112387712,X95xlpQ,0,FEM,26.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,NaN
2825975,1112351932,QNr5j54,1,MASC,29.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN


In [10]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
9199434,1112179527,6rZYzvL,0,MASC,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
7404489,1112467018,604GmM,0,MASC,40.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.0
10456274,1112289103,JBr9ALJ,1,FEM,20.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
17673340,1112261978,X9oOE9Q,0,MASC,41.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.0
16074590,1112437366,NRY9EB,1,FEM,41.0,-,-,-,Graduado,-,...,False,True,False,False,False,False,False,False,False,1.0
833429,1112033906,DrX3LdW,0,FEM,19.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.0
7872022,1112296457,EmQMr9,0,MASC,30.0,-,-,-,-,-,...,False,True,False,False,False,False,True,False,True,0.0
14457304,1112312682,0zBv1Bv,1,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
17191305,1112266518,JBrEl3r,0,FEM,38.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,0.0
17524873,1112362652,96ZNkOp,0,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.0


In [11]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0

Train:  16631472 Test:  1847942


In [12]:
gc.collect()

156

In [13]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=6000,max_depth=13,n_estimators=3000,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [14]:
score=c.score(x_test,y_test)*100
print(score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


78.6032786743307


In [15]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV8.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV8.pkl']

In [16]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
55412,29523,1112372181,Z2WE8z,FEM,38.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
90238,56087,1112448346,5ODK6M,FEM,56.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,0.0
12708,18374,1112213148,akjK4e5,MASC,19.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,2.0
18999,77029,1112464306,6rLNvPR,MASC,22.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
42036,48574,1112433301,YkwD8X,FEM,30.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
31834,70377,1112461769,NAQOaM,FEM,41.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
92155,91133,1112470393,5POB3X,FEM,33.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,False,True,False,0.0
96580,96211,1112493250,Y9L4PJ,MASC,32.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
57194,6501,1111647536,5ZaZG,MASC,43.0,-,En Curso,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
56143,5753,1111586823,199NBE,MASC,30.0,-,-,-,-,-,...,False,False,False,False,False,True,False,False,False,0.0


In [17]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.85018134, 0.14981866],
       [0.73992104, 0.26007896],
       [0.74209626, 0.25790374],
       ...,
       [0.94863247, 0.05136753],
       [0.80231836, 0.19768164],
       [0.98182767, 0.01817233]])

In [18]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [19]:
prediccion['sepostulo']=y_final2

In [20]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.149819
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.260079
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.257904
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.057610
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.196804


In [21]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.149819
1,1,0.260079
2,2,0.257904
3,3,0.057610
4,4,0.196804


In [22]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion22.csv",index=False)

In [23]:
prediccion.sepostulo.mean()

0.49580879962828034